In [1]:
#!/usr/bin/env python3

import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
import sklearn.datasets
import sklearn.model_selection
import matplotlib.pyplot as plt

%matplotlib inline

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [16]:
##MODEL PARAMETERS
###TUNABLE PARAMETERS
###imbalance ratio. set to 1.0 for balanced data
imbalance_ratio = 1.0
###ensemble size
ensemble_size = 200
###set to None to disable class weights on decision trees
tree_class_weight = None#"balanced"
###which evaluation metric to use. choose between "AUPRC", "AUROC", and "Log loss"

#toggle decision tree split optimization criterion
criterion = True

##SET RANDOM SEED
random_state = np.random.RandomState(1973)

parameters = {
    "random_state": random_state,
    "verbose": 10, #used to keep track of training progress
    "oob_score": False, #disabled to speed up training for demonstration
    "n_estimators": ensemble_size,
    "criterion": "gini" if criterion else "entropy",
    "class_weight": tree_class_weight,
}
param_sets = (
    ("Decision trees", {
        "max_features": 1, #NOTE: this should not be done in practice, only set for demonstrational purposed
    }),
    ("Decision stumps", {
        "max_depth": 1,
        "max_features": 1,
    }),
)


In [ ]:
##DATA PARAMETERS
#ratio of first class to second is set to imbalance_ratio 
class_weights = np.array([imbalance_ratio,1.0],dtype=float)
class_weights = (class_weights/np.sum(class_weights)).tolist()

#other parameters
# NOTE: if n_informative == 35, then there are no uninformative features and the forest of
# stumps performs much better
data_params = {
    "random_state": random_state,
    "n_samples": 10000,
    "n_features": 40,
    "n_redundant": 5,
    "n_informative": 34, #this leaves one uninformative feature
    "n_classes": 2,
    "weights": class_weights,
}

split_params = {
    "random_state": random_state,
    "test_size": 0.2,
}

##GENERATE RANDOM TRAINING DATA
X, y = sklearn.datasets.make_classification(**data_params)
#training/test split. stratify split by class frequencies
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, stratify=y, **split_params)

##MODELS
models = dict((
    name, 
    {"classifier": sklearn.ensemble.RandomForestClassifier(**parameters, **tree_parameters)},
) for name, tree_parameters in param_sets)

##TRAINING
for name, model in models.items():
    #train model
    model["classifier"].fit(X_train, y_train)
    #predict labels for increasing ensemble sizes
    model["predict_proba"] = np.hstack([
        tree.predict_proba(X_test)[:,1].reshape(-1,1) for tree in model["classifier"].estimators_
    ])
    model["mean_predict_probas"] = np.hstack([
        np.mean(model["predict_proba"][:,:a],axis=1).reshape(-1,1) for a in range(1, model["classifier"].n_estimators+1)
    ])
    #scipy.stats.entropy(model["predict_proba"].T)
    #evaluate prediction performance
    model["AUROC"] = np.array([
        sklearn.metrics.roc_auc_score(y_test, subforest) for subforest in model["mean_predict_probas"].T
    ])
    model["AUPRC"] = np.array([
        sklearn.metrics.auc(*(
            sklearn.metrics.precision_recall_curve(y_test, subforest)[:-1][::-1]
        )) for subforest in model["mean_predict_probas"].T
    ])
    model["Log loss"] = np.array([
            sklearn.metrics.log_loss(y_test, subforest) for subforest in model["mean_predict_probas"].T
    ])


In [ ]:
#AU: area under
#ROC: receiver operating characteristic curve
#PRC: precision-recall curve
plot_this = "AUPRC"

##PLOTTING
for name, model in models.items():
    plt.plot(model[plot_this], marker=".", label=name)

#plot labels
if plot_this[:2] == "AU":
    if plot_this == "AUROC":
        plt.axhline(y=0.5, linestyle=":", label="Baseline")
        plt.ylabel("AUROC")
    elif plot_this == "AUPRC":
        plt.axhline(y=np.min(class_weights), linestyle=":", label="Baseline")
        plt.ylabel("AUPRC")
else:
    plt.ylabel("Log loss")

plt.xlabel("Size of forest")
plt.legend()
plt.show()
